In [46]:
import gzip
import json
import os
import pandas as pd
import plotly.express as px
import progressbar
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
pd.set_option('display.max_colwidth', None)

In [20]:
path = r'../../../../src/data/LocalBusiness/'
file = r'MFile_cleaned_withLBname'
pathFile = path + file

In [21]:
dataList = []

with gzip.open(pathFile, 'r') as dataFile:
    for line in dataFile:
        lineData = json.loads(line.decode('utf-8'))
        dataList.append(lineData)
data = pd.DataFrame(dataList)

In [39]:
data.head()


,row_id,name,address,page_url,addressregion,streetaddress,addresscountry,addresslocality,postalcode,table_id,...,phone_object,E.164 format,telephoneNorm,indexValue,MatchingNumbers,distance,MatchingNumbers_New,cluster_id,clusterNtables,tableNclusters
0,5860,Le Méridien Visconti Rome,"{'addresslocality': 'Rome', 'postalcode': '001...",https://whattoexpect.marriott.com/rommd,None,VIA FEDERICO CESI 37,IT,ROME,00193,Hotel_marriott.com_September2020.json.gz,...,"{'country_code': 39, 'extension': None, 'natio...",39063684,39063684,591585,"[591585, 589400, 587459]","[-1, 0.0, 0.0]","[591585, 589400, 587459]",1,2,3096
1,5229,Residence Inn Boston Franklin,"{'addressregion': 'Massachusetts', 'postalcode...",https://www.marriott.com/hotels/local-things-t...,Massachusetts,4 FORGE PARKWAY,US,FRANKLIN,02038,Hotel_marriott.com_September2020.json.gz,...,"{'country_code': 1, 'extension': None, 'nation...",15085418188,15085418188,591212,"[591212, 585732, 602392]","[-1, 0.0, 0.0]","[591212, 585732, 602392]",4342,3,3096
2,5971,Residence Inn Boston Foxborough,"{'addresslocality': 'Foxborough', 'addressregi...",https://www.marriott.com/hotels/travel/bosfx-r...,Massachusetts,250 FOXBOROUGH BLVD,US,FOXBOROUGH,02035,Hotel_marriott.com_September2020.json.gz,...,"{'country_code': 1, 'extension': None, 'nation...",15086982800,15086982800,591654,"[587766, 591654, 600468]","[0.0, -1, 0.0]","[587766, 591654, 600468]",4359,3,3096
3,7383,Residence Inn Boston Bridgewater,"{'postalcode': '02324', 'addressregion': 'Mass...",https://www.marriott.com/hotels/hotel-reviews/...,Massachusetts,2020 PLEASANT STREET,US,BRIDGEWATER,02324,Hotel_marriott.com_September2020.json.gz,...,"{'country_code': 1, 'extension': None, 'nation...",15086971412,15086971412,592477,"[592477, 601701]","[-1, 0.0]","[592477, 601701]",4358,2,3096
4,5552,Courtyard Boston Natick,"{'addressregion': 'Massachusetts', 'addressloc...",https://www.marriott.com/hotels/hotel-deals/bo...,Massachusetts,342 SPEEN STREET,US,NATICK,01760,Hotel_marriott.com_September2020.json.gz,...,"{'country_code': 1, 'extension': None, 'nation...",15086556100,15086556100,591406,"[591406, 587365]","[-1, 0.0]","[591406, 587365]",4354,2,3096


In [40]:
def remove_stopwords(token_vector, stopwords_list):
    return token_vector.apply(lambda token_list: [word for word in token_list if word not in stopwords_list])

In [41]:
def remove_punctuation(token_vector):
    return token_vector.apply(lambda token_list: [word for word in token_list if word not in string.punctuation])

In [42]:
def jaccard_similarity_score(original, translation):
    intersect = set(original).intersection(set(translation))
    union = set(original).union(set(translation))
    try:
        return len(intersect) / len(union)
    except ZeroDivisionError:
        return 0

## Only get those clusters with size >= 10

In [44]:
df_10 = data[data["clusterNtables"] >= 10]

In [45]:
df_10

,row_id,name,address,page_url,addressregion,streetaddress,addresscountry,addresslocality,postalcode,table_id,...,phone_object,E.164 format,telephoneNorm,indexValue,MatchingNumbers,distance,MatchingNumbers_New,cluster_id,clusterNtables,tableNclusters
14094,546,Weaver Motorsports Inc.,"{'addresscountry': 'US', 'postalcode': '27616'...",https://www.bimmershops.com/near/clayton-nc/,NC,3101 SPRING FOREST RD.,US,RALEIGH,27616,LocalBusiness_bimmershops.com_September2020.js...,...,"{'country_code': 1, 'extension': None, 'nation...",19197908811,19197908811,298478,"[40274, 88451, 72512, 70316, 97807, 298478, 16...","[0.0, 0.0, 0.0, 0.0, 0.0, -1, 0.0, 0.0, 0.0, 0...","[40274, 88451, 72512, 70316, 97807, 298478, 16...",9504,11,1234
14095,945,Lafayette German Car Repair,"{'addresscountry': 'US', 'streetaddress': '332...",https://www.bimmershops.com/near/vallejo-ca,CA,"3328 MT. DIABLO BLVD., UNIT D",US,LAFAYETTE,94549,LocalBusiness_bimmershops.com_September2020.js...,...,"{'country_code': 1, 'extension': None, 'nation...",19252843390,19252843390,298876,"[111002, 72087, 40428, 99715, 97500, 87844, 16...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[111002, 72087, 40428, 99715, 97500, 87844, 16...",9534,11,1234
14111,727,Prestige Auto Works,"{'streetaddress': '33 Massie Ct', 'addresscoun...",https://www.bimmershops.com/near/elk-grove-ca,CA,33 MASSIE CT,US,SACRAMENTO,95823,LocalBusiness_bimmershops.com_September2020.js...,...,"{'country_code': 1, 'extension': None, 'nation...",19169124099,19169124099,298659,"[97498, 72042, 70373, 40427, 273433, 87843, 16...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[97498, 72042, 70373, 40427, 273433, 87843, 16...",9456,12,1234
14119,4,Burlington Auto Works,"{'addressregion': 'ON', 'contacttype': 'Mailin...",https://ca.bimmershops.com/near/cambridge-on/,ON,6-4401 CORPORATE DRIVE,CA,BURLINGTON,L7L5T9,LocalBusiness_bimmershops.com_September2020.js...,...,"{'country_code': 1, 'extension': None, 'nation...",19053329966,19053329966,297937,"[70427, 99382, 88420, 71940, 186172, 39690, 97...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[70427, 99382, 88420, 71940, 186172, 39690, 97...",8999,13,1234
14125,417,Formula Automotive Group - Burbank,"{'addresscountry': 'US', 'postalcode': '91502'...",https://www.bimmershops.com/near/palmdale-ca,CA,416 S GLENOAKS BLVD,US,BURBANK,91502,LocalBusiness_bimmershops.com_September2020.js...,...,"{'country_code': 1, 'extension': None, 'nation...",18188483116,18188483116,298349,"[168425, 70271, 97458, 40264, 71814, 186465, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[168425, 70271, 97458, 40264, 71814, 186465, 1...",8395,12,1234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37674,36,Campaya.co.uk,"{'addresslocality': 'Horsens', 'postalcode': '...",https://www.visitassensinfo.com/assens/plan-yo...,None,ERHVERVSBYVEJ 11 NULL,DK,HORSENS,8700,LocalBusiness_visitassensinfo.com_September202...,...,"{'country_code': 45, 'extension': None, 'natio...",4578746262,4578746262,23698,"[23698, 110955, 42547, 196666, 168663, 145970,...","[-1, 50.2212448321, 2.703e-07, 114.8001405835,...","[23698, 42547, 168663, 145970, 52391, 192048, ...",571,15,15
38071,44,Campaya - Holiday house by the Danish West coast,"{'addresslocality': 'Horsens', 'postalcode': '...",https://www.naturparknissumfjord.com/nissumfjo...,None,ERHVERVSBYVEJ 11 NULL,DK,HORSENS,8700,LocalBusiness_naturparknissumfjord.com_Septemb...,...,"{'country_code': 45, 'extension': None, 'natio...",4578746262,4578746262,290061,"[23698, 110955, 42547, 196666, 168663, 145970,...","[2.703e-07, 50.2212450537, 0.0, 114.8001403188...","[23698, 42547, 168663, 145970, 52391, 192048, ...",571,15,13
38692,13,Campaya Holiday Homes,"{'addresslocality': 'Horsens', 'postalcode': '...",https://www.silkeborg.com/silkeborg/explore/ca...,None,ERHVERVSBYVEJ 11 NULL,DK,HORSENS,8700,LocalBusiness_silkeborg.com_September2020.json.gz,...,"{'country_code': 45, 'extension': None, 'natio

In [53]:
#use tokenizer for lb names to get tokens for training the model
df_10['business_tokenized'] = df_10['name'].apply(lambda row: word_tokenize(row))
df_10['business_tokenized'] = remove_stopwords(df_10['business_tokenized'],stopwords.words())
df_10['business_tokenized'] = remove_punctuation (df_10['business_tokenized'])
#get tagged words
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(df_10['business_tokenized'])]
# build model and vocabulary
model = Doc2Vec(vector_size=50, min_count = 2, epochs = 25, dm = 0)
model.build_vocab(tagged_data)
# Train model
model.train(tagged_data, total_examples=model.corpus_count, epochs=25)

<ipython-input-53-f7b5743c7c96>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['business_tokenized'] = df_10['name'].apply(lambda row: word_tokenize(row))
<ipython-input-53-f7b5743c7c96>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['business_tokenized'] = remove_stopwords(df_10['business_tokenized'],stopwords.words())
<ipython-input-53-f7b5743c7c96>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [56]:
tagged_data[:10]

[TaggedDocument(words=['Weaver', 'Motorsports', 'Inc'], tags=['0']),
 TaggedDocument(words=['Lafayette', 'German', 'Car', 'Repair'], tags=['1']),
 TaggedDocument(words=['Prestige', 'Auto', 'Works'], tags=['2']),
 TaggedDocument(words=['Burlington', 'Auto', 'Works'], tags=['3']),
 TaggedDocument(words=['Formula', 'Automotive', 'Group', 'Burbank'], tags=['4']),
 TaggedDocument(words=['Uptown', 'Auto', 'Specialist'], tags=['5']),
 TaggedDocument(words=['19th', 'Street', 'Auto'], tags=['6']),
 TaggedDocument(words=['Eurotech', 'Car', 'Care', 'Center'], tags=['7']),
 TaggedDocument(words=['R.A.I', 'Motorsport'], tags=['8']),
 TaggedDocument(words=['Reliable', 'Import', 'Service'], tags=['9'])]

In [54]:
df_10.head()

,row_id,name,address,page_url,addressregion,streetaddress,addresscountry,addresslocality,postalcode,table_id,...,E.164 format,telephoneNorm,indexValue,MatchingNumbers,distance,MatchingNumbers_New,cluster_id,clusterNtables,tableNclusters,business_tokenized
14094,546,Weaver Motorsports Inc.,"{'addresscountry': 'US', 'postalcode': '27616', 'streetaddress': '3101 Spring Forest Rd.', 'addressregion': 'NC', 'contacttype': 'Mailing address', 'addresslocality': 'Raleigh'}",https://www.bimmershops.com/near/clayton-nc/,NC,3101 SPRING FOREST RD.,US,RALEIGH,27616,LocalBusiness_bimmershops.com_September2020.json.gz,...,19197908811,19197908811,298478,"[40274, 88451, 72512, 70316, 97807, 298478, 168405, 153074, 99553, 350998, 301025]","[0.0, 0.0, 0.0, 0.0, 0.0, -1, 0.0, 0.0, 0.0, 0.0, 0.0]","[40274, 88451, 72512, 70316, 97807, 298478, 168405, 153074, 99553, 350998, 301025]",9504,11,1234,"[Weaver, Motorsports, Inc]"
14095,945,Lafayette German Car Repair,"{'addresscountry': 'US', 'streetaddress': '3328 Mt. Diablo Blvd., Unit D', 'contacttype': 'Mailing address', 'addressregion': 'CA', 'postalcode': '94549', 'addresslocality': 'Lafayette'}",https://www.bimmershops.com/near/vallejo-ca,CA,"3328 MT. DIABLO BLVD., UNIT D",US,LAFAYETTE,94549,LocalBusiness_bimmershops.com_September2020.json.gz,...,19252843390,19252843390,298876,"[111002, 72087, 40428, 99715, 97500, 87844, 168343, 186354, 273262, 301053, 298876]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1]","[111002, 72087, 40428, 99715, 97500, 87844, 168343, 186354, 273262, 301053, 298876]",9534,11,1234,"[Lafayette, German, Car, Repair]"
14111,727,Prestige Auto Works,"{'streetaddress': '33 Massie Ct', 'addresscountry': 'US', 'postalcode': '95823', 'contacttype': 'Mailing address', 'addressregion': 'CA', 'addresslocality': 'Sacramento'}",https://www.bimmershops.com/near/elk-grove-ca,CA,33 MASSIE CT,US,SACRAMENTO,95823,LocalBusiness_bimmershops.com_September2020.json.gz,...,19169124099,19169124099,298659,"[97498, 72042, 70373, 40427, 273433, 87843, 168504, 186488, 110999, 99630, 298659, 301430]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, 0.0]","[97498, 72042, 70373, 40427, 273433, 87843, 168504, 186488, 110999, 99630, 298659, 301430]",9456,12,1234,"[Prestige, Auto, Works]"
14119,4,Burlington Auto Works,"{'addressregion': 'ON', 'contacttype': 'Mailing address', 'streetaddress': '6-4401 Corporate Drive', 'addresslocality': 'Burlington', 'postalcode': 'L7L5T9', 'addresscountry': 'CA'}",https://ca.bimmershops.com/near/cambridge-on/,ON,6-4401 CORPORATE DRIVE,CA,BURLINGTON,L7L5T9,LocalBusiness_bimmershops.com_September2020.json.gz,...,19053329966,19053329966,297937,"[70427, 99382, 88420, 71940, 186172, 39690, 97179, 153150, 168327, 297937, 273073, 300925, 350863]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, 0.0, 0.0, 0.0]","[70427, 99382, 88420, 71940, 186172, 39690, 97179, 153150, 168327, 297937, 273073, 300925, 350863]",8999,13,1234,"[Burlington, Auto, Works]"
14125,417,Formula Automotive Group - Burbank,"{'addresscountry': 'US', 'postalcode': '91502', 'addresslocality': 'Burbank', 'streetaddress': '416 S Glenoaks Blvd', 'contacttype': 'Mailing address', 'addressregion': 'CA'}",https://www.bimmershops.com/near/palmdale-ca,CA,416 S GLENOAKS BLVD,US,BURBANK,91502,LocalBusiness_bimmershops.com_September2020.json.gz,...,18188483116,18188483116,298349,"[168425, 70271, 97458, 40264, 71814, 186465, 111011, 153100, 301130, 298349, 273299, 350819]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, 0.0, 0.0]","[168425, 70271, 97458, 40264, 71814, 186465, 111011, 153100, 301130, 298349, 273299, 350819]",8395,12,1234,"[Formula, Automotive, Group, Burbank]"


In [ ]:
#get cluster ids for basline local businesses and with that indices of top businesses to use model
#1524820,47566,6076,14418,28307,33570,39040,51314,99153,215254,685416, 984421 , 1808651,2887810,34506065,47841827,620473,56116,94055, 150211,182246, 516888, 562955 
top_clusters_list = df_10['cluster_id'].tolist()
index_top_clusters_list=[]
for i_d in top_clusters_list:
    index_top_clusters_list.append(df_compare_electronics[df_compare_electronics['cluster_id']==id].index[0])

In [ ]:
# get most similar products for each of the base clusters and save them if they have more than 5 tables
index_top_clusters_list = [14095]
electronics_clusters_search=[]
for i in index_top_clusters_list:
    similar_doc = model.docvecs.most_similar(f'{i}', topn = 20)
    electronics_clusters_search.append(int(i))
    for index, similarity in similar_doc:
        if df_compare_electronics.iloc[int(index)]['Amount']>12:
            electronics_clusters_search.append(int(index))
    jaccard_score = df_compare_electronics['product_tokes'].apply(lambda row: jaccard_similarity_score(row,df_compare_electronics.iloc[int(i)]['product_tokes']) )
    indizes=sorted(range(len(jaccard_score)), key=lambda i: jaccard_score[i])[-20:]
    for index in indizes:
         if df_compare_electronics.iloc[int(index)]['Amount']>12:
            electronics_clusters_search.append(int(index))    
df_electroncis_final = df_compare_electronics.iloc[electronics_clusters_search]

In [60]:
model.docvecs.most_similar('0', topn = 20)

[('563', 0.9915750026702881),
 ('3000', 0.9911356568336487),
 ('31', 0.9909439086914062),
 ('1477', 0.990917444229126),
 ('1025', 0.9908275604248047),
 ('1664', 0.9907021522521973),
 ('919', 0.9906821250915527),
 ('2151', 0.9905506372451782),
 ('460', 0.9905164241790771),
 ('3104', 0.9903356432914734),
 ('2827', 0.9901047945022583),
 ('2469', 0.9899241328239441),
 ('171', 0.9898597002029419),
 ('2607', 0.9897487759590149),
 ('3099', 0.9897010922431946),
 ('193', 0.9896090030670166),
 ('56', 0.9895272850990295),
 ('840', 0.9895025491714478),
 ('2118', 0.9894989728927612),
 ('1900', 0.989498496055603)]